Combinacion de modelos 

In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np

print("Librerías importadas correctamente")
print("Combinación: Primeros 100 AutoGluon, siguientes 200 Regresión Lineal, resto Media 12 093")

Librerías importadas correctamente
Combinación: Primeros 100 AutoGluon, siguientes 200 Regresión Lineal, resto Media 12 093


In [2]:
# Cargar los tres archivos de predicciones
print("Cargando archivos de predicciones...")

# Cargar predicciones de AutoGluon
pred_autogluon = pd.read_csv('data/pred_autogluon_01.csv')
print(f"AutoGluon - Shape: {pred_autogluon.shape}")
print("AutoGluon - Primeras filas:")
print(pred_autogluon.head())

print("\n" + "="*50 + "\n")

# Cargar predicciones de Regresión Lineal
pred_regresion = pd.read_csv('data/pred_regresion_lineal01.csv')
print(f"Regresión Lineal - Shape: {pred_regresion.shape}")
print("Regresión Lineal - Primeras filas:")
print(pred_regresion.head())

print("\n" + "="*50 + "\n")

# Cargar predicciones de Media 12 093
pred_media = pd.read_csv('data/pred_medias_12_093.csv')
print(f"Media 12 093 - Shape: {pred_media.shape}")
print("Media 12 093 - Primeras filas:")
print(pred_media.head())

# Verificar que todos los archivos tienen product_ids ordenados
print(f"\nProduct_id mínimo AutoGluon: {pred_autogluon['product_id'].min()}")
print(f"Product_id máximo AutoGluon: {pred_autogluon['product_id'].max()}")
print(f"Product_id mínimo Regresión Lineal: {pred_regresion['product_id'].min()}")
print(f"Product_id máximo Regresión Lineal: {pred_regresion['product_id'].max()}")
print(f"Product_id mínimo Media 12 093: {pred_media['product_id'].min()}")
print(f"Product_id máximo Media 12 093: {pred_media['product_id'].max()}")

Cargando archivos de predicciones...
AutoGluon - Shape: (780, 2)
AutoGluon - Primeras filas:
   product_id           tn
0       20001  1322.782727
1       20002  1076.158858
2       20003   696.172870
3       20004   519.217793
4       20005   501.010270


Regresión Lineal - Shape: (780, 2)
Regresión Lineal - Primeras filas:
   product_id           tn
0       20001  1162.707525
1       20002  1183.640604
2       20003   684.763931
3       20004   580.484961
4       20005   563.560780


Media 12 093 - Shape: (780, 2)
Media 12 093 - Primeras filas:
   product_id           tn
0       20001  1352.901430
1       20002  1093.156542
2       20003   730.028059
3       20004   583.310255
4       20005   621.491197

Product_id mínimo AutoGluon: 20001
Product_id máximo AutoGluon: 21276
Product_id mínimo Regresión Lineal: 20001
Product_id máximo Regresión Lineal: 21276
Product_id mínimo Media 12 093: 20001
Product_id máximo Media 12 093: 21276


In [3]:
# Ordenar los datos por product_id para garantizar consistencia
print("Ordenando datos por product_id...")

# Ordenar los tres dataframes por product_id
pred_autogluon = pred_autogluon.sort_values('product_id').reset_index(drop=True)
pred_regresion = pred_regresion.sort_values('product_id').reset_index(drop=True)
pred_media = pred_media.sort_values('product_id').reset_index(drop=True)

print("Datos ordenados correctamente")

# Verificar que todos los archivos tienen los mismos product_ids
productos_autogluon = set(pred_autogluon['product_id'])
productos_regresion = set(pred_regresion['product_id'])
productos_media = set(pred_media['product_id'])

if productos_autogluon == productos_regresion == productos_media:
    print("✅ Los tres archivos tienen exactamente los mismos productos")
    productos_totales = len(productos_autogluon)
    print(f"Total de productos disponibles: {productos_totales}")
else:
    print("⚠️ Los archivos NO tienen exactamente los mismos productos")
    productos_comunes = productos_autogluon & productos_regresion & productos_media
    print(f"Productos comunes: {len(productos_comunes)}")
    
print(f"\nPrimeros 5 product_ids ordenados:")
print(pred_autogluon['product_id'].head().tolist())

Ordenando datos por product_id...
Datos ordenados correctamente
✅ Los tres archivos tienen exactamente los mismos productos
Total de productos disponibles: 780

Primeros 5 product_ids ordenados:
[20001, 20002, 20003, 20004, 20005]


In [4]:
# Crear la combinación de tres modelos con distribución específica
print("Creando combinación de tres modelos...")

# Definir los puntos de corte
corte_autogluon = 100  # Primeros 100 productos
corte_regresion = 300  # Productos 101-300 (200 productos)
total_productos = len(pred_autogluon)

print(f"Estrategia de combinación:")
print(f"  - Productos 1-{corte_autogluon}: AutoGluon")
print(f"  - Productos {corte_autogluon+1}-{corte_regresion}: Regresión Lineal")
print(f"  - Productos {corte_regresion+1}-{total_productos}: Media 12 093")

# Tomar los primeros 100 productos de AutoGluon
primeros_100_autogluon = pred_autogluon.head(corte_autogluon).copy()
primeros_100_autogluon['origen'] = 'AutoGluon'

# Tomar los productos 101-300 de Regresión Lineal
productos_101_300_regresion = pred_regresion.iloc[corte_autogluon:corte_regresion].copy()
productos_101_300_regresion['origen'] = 'Regresion_Lineal'

# Tomar los productos restantes (301+) de Media 12 093
productos_restantes_media = pred_media.iloc[corte_regresion:].copy()
productos_restantes_media['origen'] = 'Media_12_093'

print(f"\nPrimeros 100 productos (AutoGluon):")
print(f"  Shape: {primeros_100_autogluon.shape}")
print(f"  Product_id rango: {primeros_100_autogluon['product_id'].min()} - {primeros_100_autogluon['product_id'].max()}")

print(f"\nProductos 101-300 (Regresión Lineal):")
print(f"  Shape: {productos_101_300_regresion.shape}")
print(f"  Product_id rango: {productos_101_300_regresion['product_id'].min()} - {productos_101_300_regresion['product_id'].max()}")

print(f"\nProductos restantes (Media 12 093):")
print(f"  Shape: {productos_restantes_media.shape}")
print(f"  Product_id rango: {productos_restantes_media['product_id'].min()} - {productos_restantes_media['product_id'].max()}")

# Combinar los tres conjuntos
combinacion_final = pd.concat([primeros_100_autogluon, productos_101_300_regresion, productos_restantes_media], ignore_index=True)

print(f"\nCombinación final:")
print(f"  Shape total: {combinacion_final.shape}")
print(f"  Productos de AutoGluon: {(combinacion_final['origen'] == 'AutoGluon').sum()}")
print(f"  Productos de Regresión Lineal: {(combinacion_final['origen'] == 'Regresion_Lineal').sum()}")
print(f"  Productos de Media 12 093: {(combinacion_final['origen'] == 'Media_12_093').sum()}")

Creando combinación de tres modelos...
Estrategia de combinación:
  - Productos 1-100: AutoGluon
  - Productos 101-300: Regresión Lineal
  - Productos 301-780: Media 12 093

Primeros 100 productos (AutoGluon):
  Shape: (100, 3)
  Product_id rango: 20001 - 20114

Productos 101-300 (Regresión Lineal):
  Shape: (200, 3)
  Product_id rango: 20116 - 20365

Productos restantes (Media 12 093):
  Shape: (480, 3)
  Product_id rango: 20366 - 21276

Combinación final:
  Shape total: (780, 3)
  Productos de AutoGluon: 100
  Productos de Regresión Lineal: 200
  Productos de Media 12 093: 480


In [5]:
# Mostrar ejemplos de las transiciones entre modelos
print("ANÁLISIS DE LAS TRANSICIONES ENTRE MODELOS")
print("="*50)

# Mostrar los últimos productos de AutoGluon y los primeros de Regresión Lineal
print("Últimos 5 productos de AutoGluon:")
transicion_autogluon = combinacion_final[combinacion_final['origen'] == 'AutoGluon'].tail()
print(transicion_autogluon[['product_id', 'tn', 'origen']])

print(f"\nPrimeros 5 productos de Regresión Lineal:")
transicion_regresion = combinacion_final[combinacion_final['origen'] == 'Regresion_Lineal'].head()
print(transicion_regresion[['product_id', 'tn', 'origen']])

print(f"\nÚltimos 5 productos de Regresión Lineal:")
transicion_regresion_final = combinacion_final[combinacion_final['origen'] == 'Regresion_Lineal'].tail()
print(transicion_regresion_final[['product_id', 'tn', 'origen']])

print(f"\nPrimeros 5 productos de Media 12 093:")
transicion_media = combinacion_final[combinacion_final['origen'] == 'Media_12_093'].head()
print(transicion_media[['product_id', 'tn', 'origen']])

# Comparar estadísticas entre los tres grupos
print(f"\nESTADÍSTICAS COMPARATIVAS:")
stats_autogluon = combinacion_final[combinacion_final['origen'] == 'AutoGluon']['tn']
stats_regresion = combinacion_final[combinacion_final['origen'] == 'Regresion_Lineal']['tn']
stats_media = combinacion_final[combinacion_final['origen'] == 'Media_12_093']['tn']

print(f"\nPrimeros 100 productos (AutoGluon):")
print(f"  Promedio: {stats_autogluon.mean():.2f}")
print(f"  Mediana:  {stats_autogluon.median():.2f}")
print(f"  Std:      {stats_autogluon.std():.2f}")
print(f"  Min:      {stats_autogluon.min():.2f}")
print(f"  Max:      {stats_autogluon.max():.2f}")

print(f"\nProductos 101-300 (Regresión Lineal):")
print(f"  Promedio: {stats_regresion.mean():.2f}")
print(f"  Mediana:  {stats_regresion.median():.2f}")
print(f"  Std:      {stats_regresion.std():.2f}")
print(f"  Min:      {stats_regresion.min():.2f}")
print(f"  Max:      {stats_regresion.max():.2f}")

print(f"\nProductos 301+ (Media 12 093):")
print(f"  Promedio: {stats_media.mean():.2f}")
print(f"  Mediana:  {stats_media.median():.2f}")
print(f"  Std:      {stats_media.std():.2f}")
print(f"  Min:      {stats_media.min():.2f}")
print(f"  Max:      {stats_media.max():.2f}")

print(f"\nCombinación completa:")
print(f"  Promedio: {combinacion_final['tn'].mean():.2f}")
print(f"  Mediana:  {combinacion_final['tn'].median():.2f}")
print(f"  Std:      {combinacion_final['tn'].std():.2f}")

ANÁLISIS DE LAS TRANSICIONES ENTRE MODELOS
Últimos 5 productos de AutoGluon:
    product_id         tn     origen
95       20108  62.664800  AutoGluon
96       20109  69.516097  AutoGluon
97       20111  76.556917  AutoGluon
98       20112  85.717933  AutoGluon
99       20114  60.980347  AutoGluon

Primeros 5 productos de Regresión Lineal:
     product_id         tn            origen
100       20116  63.698803  Regresion_Lineal
101       20117  46.876846  Regresion_Lineal
102       20118  48.949437  Regresion_Lineal
103       20119  45.141238  Regresion_Lineal
104       20120  59.513806  Regresion_Lineal

Últimos 5 productos de Regresión Lineal:
     product_id         tn            origen
295       20359  13.298528  Regresion_Lineal
296       20361  18.273224  Regresion_Lineal
297       20362   8.765510  Regresion_Lineal
298       20364  23.736458  Regresion_Lineal
299       20365  12.901056  Regresion_Lineal

Primeros 5 productos de Media 12 093:
     product_id         tn        ori

In [6]:
# Preparar y guardar el archivo final
print("Preparando archivo final...")

# Crear el resultado final con solo product_id y tn (sin la columna 'origen')
resultado_final = combinacion_final[['product_id', 'tn']].copy()

# Verificar que está ordenado por product_id
resultado_final = resultado_final.sort_values('product_id').reset_index(drop=True)

# Verificar que no hay valores nulos
print(f"Valores nulos en el resultado final: {resultado_final.isnull().sum().sum()}")
print(f"Shape del resultado final: {resultado_final.shape}")

print(f"\nPrimeras 10 filas del resultado final:")
print(resultado_final.head(10))

print(f"\nÚltimas 5 filas del resultado final:")
print(resultado_final.tail())

# Verificar continuidad en los product_ids
print(f"\nVerificación de continuidad:")
print(f"Product_id mínimo: {resultado_final['product_id'].min()}")
print(f"Product_id máximo: {resultado_final['product_id'].max()}")
print(f"Total de productos: {len(resultado_final)}")

# Puntos de transición
producto_100 = resultado_final.iloc[99]   # índice 99 = producto 100
producto_101 = resultado_final.iloc[100]  # índice 100 = producto 101
producto_300 = resultado_final.iloc[299]  # índice 299 = producto 300
producto_301 = resultado_final.iloc[300]  # índice 300 = producto 301

print(f"\nPuntos de transición:")
print(f"Producto {corte_autogluon} (último de AutoGluon): ID {producto_100['product_id']}, tn = {producto_100['tn']:.2f}")
print(f"Producto {corte_autogluon+1} (primero de Regresión): ID {producto_101['product_id']}, tn = {producto_101['tn']:.2f}")
print(f"Producto {corte_regresion} (último de Regresión): ID {producto_300['product_id']}, tn = {producto_300['tn']:.2f}")
print(f"Producto {corte_regresion+1} (primero de Media): ID {producto_301['product_id']}, tn = {producto_301['tn']:.2f}")

Preparando archivo final...
Valores nulos en el resultado final: 0
Shape del resultado final: (780, 2)

Primeras 10 filas del resultado final:
   product_id           tn
0       20001  1322.782727
1       20002  1076.158858
2       20003   696.172870
3       20004   519.217793
4       20005   501.010270
5       20006   451.382664
6       20007   397.499986
7       20008   389.824683
8       20009   490.563595
9       20010   383.118202

Últimas 5 filas del resultado final:
     product_id        tn
775       21263  0.027894
776       21265  0.083273
777       21266  0.088033
778       21267  0.086337
779       21276  0.042266

Verificación de continuidad:
Product_id mínimo: 20001
Product_id máximo: 21276
Total de productos: 780

Puntos de transición:
Producto 100 (último de AutoGluon): ID 20114.0, tn = 60.98
Producto 101 (primero de Regresión): ID 20116.0, tn = 63.70
Producto 300 (último de Regresión): ID 20365.0, tn = 12.90
Producto 301 (primero de Media): ID 20366.0, tn = 11.86


In [7]:
# Guardar el archivo final
archivo_salida = "data/pred_combina_autogluon100_rl_200_media_12_093.csv"

print(f"Guardando combinación de modelos en: {archivo_salida}")

# Crear directorio data si no existe
import os
os.makedirs("data", exist_ok=True)

# Guardar el archivo
resultado_final.to_csv(archivo_salida, index=False)

print(f"✅ Archivo guardado exitosamente: {archivo_salida}")
print(f"Total de productos: {len(resultado_final)}")

# Verificar que el archivo se guardó correctamente
if os.path.exists(archivo_salida):
    tamaño_archivo = os.path.getsize(archivo_salida)
    print(f"Tamaño del archivo: {tamaño_archivo} bytes")
    
    # Leer las primeras y últimas líneas del archivo guardado para verificar
    verificacion_inicio = pd.read_csv(archivo_salida, nrows=5)
    print(f"\nVerificación - Primeras 5 líneas del archivo guardado:")
    print(verificacion_inicio)
    
    # Verificar los productos cerca de los puntos de transición
    verificacion_completa = pd.read_csv(archivo_salida)
    print(f"\nVerificación de transiciones:")
    print("Transición AutoGluon → Regresión Lineal (productos 98-103):")
    transicion1 = verificacion_completa.iloc[97:103]  # productos 98-103
    print(transicion1)
    
    print("\nTransición Regresión Lineal → Media 12 093 (productos 298-303):")
    transicion2 = verificacion_completa.iloc[297:303]  # productos 298-303
    print(transicion2)
    
else:
    print("⚠️ Error: No se pudo crear el archivo")

Guardando combinación de modelos en: data/pred_combina_autogluon100_rl_200_media_12_093.csv
✅ Archivo guardado exitosamente: data/pred_combina_autogluon100_rl_200_media_12_093.csv
Total de productos: 780
Tamaño del archivo: 17802 bytes

Verificación - Primeras 5 líneas del archivo guardado:
   product_id           tn
0       20001  1322.782727
1       20002  1076.158858
2       20003   696.172870
3       20004   519.217793
4       20005   501.010270

Verificación de transiciones:
Transición AutoGluon → Regresión Lineal (productos 98-103):
     product_id         tn
97        20111  76.556917
98        20112  85.717933
99        20114  60.980347
100       20116  63.698803
101       20117  46.876846
102       20118  48.949437

Transición Regresión Lineal → Media 12 093 (productos 298-303):
     product_id         tn
297       20362   8.765510
298       20364  23.736458
299       20365  12.901056
300       20366  11.859593
301       20367  11.330856
302       20368  17.731964


In [8]:
# Resumen final de la combinación
print("RESUMEN FINAL DE LA COMBINACIÓN DE TRES MODELOS")
print("="*60)
print(f"📁 Archivo fuente 1:        data/pred_autogluon_01.csv (primeros 100)")
print(f"📁 Archivo fuente 2:        data/pred_regresion_lineal01.csv (101-300)")
print(f"📁 Archivo fuente 3:        data/pred_medias_12_093.csv (301+)")
print(f"📁 Archivo generado:        {archivo_salida}")
print(f"📊 Total de productos:      {len(resultado_final)}")

print(f"\n🔄 ESTRATEGIA DE COMBINACIÓN:")
print(f"   • Productos 1-{corte_autogluon}:      AutoGluon puro")
print(f"   • Productos {corte_autogluon+1}-{corte_regresion}:    Regresión Lineal")
print(f"   • Productos {corte_regresion+1}-{len(resultado_final)}:      Media 12 093")

print(f"\n📈 ESTADÍSTICAS POR SEGMENTO:")
print(f"   AutoGluon (1-{corte_autogluon}):")
print(f"     - Promedio: {stats_autogluon.mean():.2f}")
print(f"     - Rango: {stats_autogluon.min():.2f} - {stats_autogluon.max():.2f}")

print(f"   Regresión Lineal ({corte_autogluon+1}-{corte_regresion}):")
print(f"     - Promedio: {stats_regresion.mean():.2f}")
print(f"     - Rango: {stats_regresion.min():.2f} - {stats_regresion.max():.2f}")

print(f"   Media 12 093 ({corte_regresion+1}-{len(resultado_final)}):")
print(f"     - Promedio: {stats_media.mean():.2f}")
print(f"     - Rango: {stats_media.min():.2f} - {stats_media.max():.2f}")

print(f"\n🎯 JUSTIFICACIÓN:")
print(f"   Esta combinación de tres modelos permite diversificar:")
print(f"   - AutoGluon: ML avanzado para los primeros productos")
print(f"   - Regresión Lineal: Enfoque estadístico para el segmento medio")
print(f"   - Media 12 093: Estabilidad histórica para el resto")
print(f"   Creando un enfoque híbrido que maximiza la diversidad.")

print(f"\n✅ Combinación de tres modelos completada exitosamente!")
print(f"   El archivo {archivo_salida} está listo para usar.")

RESUMEN FINAL DE LA COMBINACIÓN DE TRES MODELOS
📁 Archivo fuente 1:        data/pred_autogluon_01.csv (primeros 100)
📁 Archivo fuente 2:        data/pred_regresion_lineal01.csv (101-300)
📁 Archivo fuente 3:        data/pred_medias_12_093.csv (301+)
📁 Archivo generado:        data/pred_combina_autogluon100_rl_200_media_12_093.csv
📊 Total de productos:      780

🔄 ESTRATEGIA DE COMBINACIÓN:
   • Productos 1-100:      AutoGluon puro
   • Productos 101-300:    Regresión Lineal
   • Productos 301-780:      Media 12 093

📈 ESTADÍSTICAS POR SEGMENTO:
   AutoGluon (1-100):
     - Promedio: 196.05
     - Rango: 43.38 - 1322.78
   Regresión Lineal (101-300):
     - Promedio: 31.33
     - Rango: 7.62 - 245.45
   Media 12 093 (301-780):
     - Promedio: 5.00
     - Rango: 0.02 - 29.08

🎯 JUSTIFICACIÓN:
   Esta combinación de tres modelos permite diversificar:
   - AutoGluon: ML avanzado para los primeros productos
   - Regresión Lineal: Enfoque estadístico para el segmento medio
   - Media 12 093: